In [3]:
import time
import re
import requests
from pyquery import PyQuery as pq
import json

In [4]:
url = 'https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&viewtype=L&CatId=&SearchText=3d+mink+eyelash'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'
headers['Accept-Encoding'] = 'gzip, deflate, br'
headers['Accept-Language'] = 'zh-CN,zh;q=0.9,en;q=0.8'
headers['Referer'] = url
headers['Connection'] = 'Keep-Alive'
headers['Host'] = 'www.alibaba.com'
headers['Upgrade-Insecure-Requests'] = '1'
# requests.session().cookies.clear()


In [40]:
t = time.time()
response = requests.get(url, headers=headers)

t0 = time.time()
result = re.search(r'_search_result_data =(.*)page.setPageData\(_search_result_data\)', response.text, re.M|re.DOTALL)
obj = json.loads(result.group(1))
items = obj['normalList']
for idx, item in enumerate(items):
    company = {}
    if(item['isBrandAd']):
#             top_sponsor = company
        pass
    elif(item['isP4p']):
#             sponsor_list.append(company)
        pass
    else:
        break

    company['years'] = item['supplierYear']
    company['name'] = item['supplierName']
    company['url'] = item['supplierHref']
    if 'record' in item:
        company['record'] = []
        if 'transaction' in item['record']:
            company['record'].append(item['record']['transaction']['num'])
            company['record'].append('Transactions(6 months)')
            company['record'].append(item['record']['transaction']['conducted'])
        company['record'].append('Response Rate')
        company['record'].append(item['record']['responseRate'])

t1 = time.time()

doc = pq(response.text)
divs = doc('.m-product-item')
for ediv in divs:
    company = {}

    div = pq(ediv)
    if div.find('span.sking'):
#         top_sponsor = company
        pass
    elif div.find('span.sl'):
#         sponsor_list.append(company)
        pass
    else:
        break

    extra = div.find('.item-extra')
    if extra:
        i = extra.find('div.stitle i')
        company['years'] = re.search('year-num(\d+)', i.attr('class')).group(1)
        a = extra.find('div.stitle a:last-child')
        company['name'] = a.text().strip()
        company['url'] = 'https:' + a.attr('href')
        record = extra.find('ul.record')
    else:
        container = div.find('div.brand-right-container')
        text = container.find('i.year-num').attr('class')
        company['years'] = re.search('year-num(\d+)', text).group(1)
        a = container.find('div.supplier a')
        company['name'] = a.text().strip()
        company['url'] = 'https:' + a.attr('href')
        record = container.find('ul.record-container')
    if record:
        company['record'] = []
        company['record'].append(record.find('li:first-child div.lab b').text().strip())
        company['record'].append('Transactions(6 months)')
        company['record'].append(record.find('li:first-child div.num').text().strip())
        company['record'].append('Response Rate')
        company['record'].append(record.find('li:last-child div.num').text().strip())
t2 = time.time()
print(t0-t)
print(t1-t0)
print(t2-t1)

1.0015697479248047
0.007996320724487305
0.07895040512084961


In [266]:

# result = re.match(r'page.setPageData\(_search_result_data\)', response.text, re.M)
obj = json.loads(result.group(1))
len()

50

True

In [240]:
req = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(req).read()
doc =  pq(response)

In [113]:
doc

[<p>]

{'years': '3', 'name': 'Yiwu Qiaoti E-Commerce Co., Ltd.', 'url': 'https://qtbeauty.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['331', 'Transactions(6 months)', '210,000+', 'Response Rate', '94.1%']}
{'years': '5', 'name': 'Tianjin Yao Poly Import And Export Limited Company', 'url': 'https://tianjinyaoju.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['185', 'Transactions(6 months)', '90,000+', 'Response Rate', '99.0%']}
{'years': '9', 'name': 'Qingdao Mermaid Hair Industry Co., Ltd.', 'url': 'https://mermaidhair.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['7', 'Transactions(6 months)', '400+', 'Response Rate', '95.9%']}
{'years': '1', 'name': 'Qingdao Lody Love Hair Products Co., Ltd.', 'url': 'https://lodyhair.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['', 'Transactions(6 months)', '100.0%', 'Response Rate', '100.0%']}
{'years': '1', 'name': 'Qingdao Xindongweitai Trading Co., Ltd.', 'url': 'https://xdvictoria.en.alibab

In [19]:
from bs4 import BeautifulSoup


In [21]:

soup = BeautifulSoup(response.text, 'html.parser')
divs = soup.find_all('div', class_='m-product-item')

for idx,div in enumerate(divs):
    # print(kws, idx)
    company = {}
    extra = div.find(class_='item-extra')
    if extra:
        a = extra.find('div', class_='stitle').a
        company['years'] = re.search('year-num(\d+)', str(a.i)).group(1)
        a = a.next_sibling.next_sibling
        company['name'] = a.string.strip()
        company['url'] = 'https:' + a['href']
        ul = extra.find('ul', class_='record')
    else:
        container = div.find('div', class_='brand-right-container')
        text = str(container.find('i', class_='year-num'))
        company['years'] = re.search('year-num(\d+)', text).group(1)
        a = container.find('div', class_='supplier').a
        company['name'] = a.string.strip()
        company['url'] = 'https:' + a['href']
        ul = container.find('ul', class_='record-container')

    if ul:
        company['record'] = [re.sub('\n', '', x).strip() for x in ul.findAll(text=True) if x != '\n']

    if div.find('span', class_='sking'):
        top_sponsor = company
    elif div.find('span', class_='sl'):
        sponsor_list.append(company)
    else:
        break
    print(company)

{'years': '3', 'name': 'Yiwu Qiaoti E-Commerce Co., Ltd.', 'url': 'https://qtbeauty.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['331', 'Transactions(6 months)', '210,000+', 'Response Rate', '94.1%']}


NameError: name 'sponsor_list' is not defined